In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import re
import requests
pd.options.display.max_rows = 5000 
import warnings
warnings.filterwarnings('ignore')

In [2]:
test_lst = ['AHT_div','AV_div','BA_div','GSK_div','HSBA_div','JD_div','NG_div','RDSB_div','RIO_div',
            'SPX_div','TSCO_div','TW_div','ULVR_div']

In [3]:
def get_years(lst, start_int, end_int):
    df = pd.read_excel('/Users/paxton615/Desktop/Stock_Price_csv/tier_1_stocks/dividend/{}.xlsx'.format(lst[0]), skiprows=1)
    df.columns = [''.join(filter(str.isalpha, i)).lower() for i in df.columns]
    df = [int(i[3:]) for i in list(df['yearend'])]
    df = list(filter(lambda i: end_int>=i>=start_int, df))
    return(df)

def get_dividends(lst, start_int, end_int):
    new_list = []
    for i in lst:
        df = pd.read_excel('/Users/paxton615/Desktop/Stock_Price_csv/tier_1_stocks/dividend/{}.xlsx'.format(i), skiprows=1)
        df.columns = [''.join(filter(str.isalpha, i)).lower() for i in df.columns]  #由于有些标题带*号，只保留字符，然后都变小写
        df = df[['yearend','total']]
        df.columns = ['year',i]
        df['year'] = [int(i[3:]) for i in list(df['year'])]
        df = df.loc[(df['year']<=end_int) & (df['year']>=start_int)] 
        df[i] = [float(re.sub("[^\d\.]", "", i)) for i in df[i]] 
        new_list.append(df[i].reset_index(drop=True))
    return(new_list)

In [9]:
stocks = ['AHT', 'AV', 'BA', 'GSK', 'HSBA', 'JD', 'NG', 'RDSB', 'RIO', 'SPX', 'TSCO', 'TW', 'ULVR']
# stocks = ['AHT', 'GSK', 'JD', 'NG', 'RDSB', 'RIO', 'SPX', 'TSCO', ]
# start_date = '2016-01-01'
# end_date = '2021-01-01'

In [10]:
def get_dates(lst, start_str, end_str):
    data = pd.read_csv(r'/Users/paxton615/Desktop/Stock_Price_csv/tier_1_stocks/{}.csv'.format(lst[0]))
    data = data.loc[(data['time']>=start_str)&(data['time']<end_str)]
    date = [datetime.fromisoformat(i[:10]) for i in data['time']]
    return(date)

def get_prices(lst,start_str, end_str):   
    new_list = []
    for i in lst:
        data = pd.read_csv(r'/Users/paxton615/Desktop/Stock_Price_csv/tier_1_stocks/{}.csv'.format(i)) # read data from csv 注意文件夹路径和文件名
        df = data[['time','close']]                                          # get time and close price
        df.columns = ['date', i]                                       # change column names
        df.date = [datetime.fromisoformat(i[:10]) for i in df['date']]       # convert dates into datetime
        df = df.loc[(df['date']>=start_str)&(df['date']<end_str)]    # make sure date start from 2016-01-01
        df = df.reset_index(drop=True)
        new_list.append(df[i])
    return(new_list)

### 5年的红利

In [13]:
div_yrs = get_years(test_lst, 2015, 2020)

divs = get_dividends(test_lst, 2015, 2020)

dividends = pd.DataFrame(np.array(divs).T, columns=test_lst , index=np.array(div_yrs).T)
dividends = dividends.iloc[::-1] #将数据从老到新排列
dividends

,AHT_div,AV_div,BA_div,GSK_div,HSBA_div,JD_div,NG_div,RDSB_div,RIO_div,SPX_div,TSCO_div,TW_div,ULVR_div
2015,15.25,20.8,20.9,80.0,51.0,1.41,42.87,188.0,215.0,69.0,1.16,1.67,120.80
2016,22.50,23.3,21.3,80.0,51.0,1.48,43.34,188.0,170.0,76.0,0.00,2.82,128.04
2017,27.50,27.4,21.8,80.0,51.0,1.55,44.27,188.0,290.0,87.5,0.00,4.74,143.40
2018,33.00,30.0,22.2,80.0,51.0,1.63,45.93,188.0,307.0,100.0,3.00,6.24,154.88
2019,40.00,15.5,23.2,80.0,30.0,1.71,47.34,188.0,382.0,110.0,5.77,3.84,164.16
2020,40.65,21.0,23.7,80.0,15.0,0.28,48.57,65.3,464.0,118.0,9.15,4.14,165.80


### 5年中每天的价格

In [11]:
date = get_dates(stocks,'2015-01-01','2020-01-01')

prices = get_prices(stocks,'2015-01-01','2020-01-01')

prices = pd.DataFrame(np.array(prices).T, columns=stocks , index=np.array(date).T)
prices.head()

,AHT,AV,BA,GSK,HSBA,JD,NG,RDSB,RIO,SPX,TSCO,TW,ULVR
2015-01-02,1165.0,484.39999,469.20001,1383.0,612.00000,101.600000,908.29999,2235.5,2970.0,2848.0,188.00000,134.89999,2606.0
2015-01-05,1182.0,470.79999,462.00000,1369.5,603.90002,101.400000,909.70001,2128.5,2883.5,2832.0,181.60001,133.80000,2573.0
2015-01-06,1109.0,464.00000,456.70001,1357.0,593.70001,94.400000,896.79999,2121.0,2944.5,2784.0,178.80000,131.10001,2574.0
2015-01-07,1109.0,463.29999,462.70001,1379.0,601.00000,95.159998,917.50000,2148.5,2962.5,2800.0,182.00000,132.39999,2591.0
2015-01-08,1131.0,483.20001,468.89999,1427.5,606.20001,97.000000,932.40002,2200.0,3027.5,2849.0,209.25000,132.80000,2668.0


#### 假设一个投资者拥有200万英镑的资产，且我们将资金在所有股票上平均分配。试计算最终的红利dividend回报是多少。

In [14]:
fund = 200*10000*100 # unit = pence

divide fund equally into 13 stocks

In [17]:
equal_portion = np.full(shape=len(stocks), fill_value=fund/len(stocks), dtype=np.float)
equal_portion

array([15384615.38461538, 15384615.38461538, 15384615.38461538,
       15384615.38461538, 15384615.38461538, 15384615.38461538,
       15384615.38461538, 15384615.38461538, 15384615.38461538,
       15384615.38461538, 15384615.38461538, 15384615.38461538,
       15384615.38461538])

calculate how many shares that each equal portion can give us

In [20]:
shares = np.round(equal_portion / prices.iloc[0], 0)
shares

AHT      13206.0
AV       31760.0
BA       32789.0
GSK      11124.0
HSBA     25138.0
JD      151423.0
NG       16938.0
RDSB      6882.0
RIO       5180.0
SPX       5402.0
TSCO     81833.0
TW      114045.0
ULVR      5904.0
Name: 2015-01-02 00:00:00, dtype: float64

client invest at the beginning of 2015 for 5 years. dividend years are 2015, 2016, 2017, 2018, 2019

In [22]:
dividends.iloc[:5].sum()

AHT_div      138.25
AV_div       117.00
BA_div       109.40
GSK_div      400.00
HSBA_div     234.00
JD_div         7.78
NG_div       223.75
RDSB_div     940.00
RIO_div     1364.00
SPX_div      442.50
TSCO_div       9.93
TW_div        19.31
ULVR_div     711.28
dtype: float64

shares * dividends.iloc[:5].sum() gives each stock dividends paid within 5 years

In [28]:
div_5yrs = np.array(shares) * np.array(dividends.iloc[:5].sum())
div_5yrs

array([1825729.5 , 3715920.  , 3587116.6 , 4449600.  , 5882292.  ,
       1178070.94, 3789877.5 , 6469080.  , 7065520.  , 2390385.  ,
        812601.69, 2202208.95, 4199397.12])

In [33]:
div_5yrs.sum() / fund / 5 # 所有分红收益相加 / 初始投资 = 投资期间总回报率， 投资期间总回报率 / 年份 = 年回报率

0.0475677993

calculate 5 years dividend ROI for each stocks

In [30]:
div_5yrs_roi = div_5yrs / equal_portion
div_5yrs_roi                            # 个股分红 / 各股初期资金分配（相当于权数） = 各股期间ROI 

array([0.11867242, 0.2415348 , 0.23316258, 0.289224  , 0.38234898,
       0.07657461, 0.24634204, 0.4204902 , 0.4592588 , 0.15537502,
       0.05281911, 0.14314358, 0.27296081])

报告中需要用到，派息率排行从高到低

In [44]:
rank_div_5yrs = pd.DataFrame([div_5yrs_roi], columns = dividends.columns).T.sort_values(by=0, ascending=False).reset_index()

In [47]:
rank_div_5yrs.index = rank_div_5yrs.index+1

In [48]:
rank_div_5yrs

,index,0
1,RIO_div,0.459259
2,RDSB_div,0.420490
3,HSBA_div,0.382349
4,GSK_div,0.289224
5,ULVR_div,0.272961
6,NG_div,0.246342
7,AV_div,0.241535
8,BA_div,0.233163
9,SPX_div,0.155375
10,TW_div,0.143144


In [36]:
div_5yrs_roi.sum()/ len(stocks) /5 # 个各股期间ROI .sum() / 股票个数 = 期间平均回报率， 期间平均回报率 / 5年 = 年回报率

0.047567799300000005

如果每只股票上投入的fund不是平均的，最正确的计算方法是 （[资金分配权数] * [各股回报率] ）.sum()就是整个Portfolio这段时间的总Return，再除以年数或者月数，就可以简单分解成年回报率了